<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# **Project 3:** Data Collection

### Data Collection
- Import The Libraries Required To Complete The Problem Statement
- Initialize PRAW and Authenicate Reddit API
- Read-In the data **(subreddit post you want to compare)**
- Set up a function that calls in the number of post required. **( 1000 post)**
- View the data (View the first few rows, and see how much data is in your dataset) **(Hint: .head(), and .shape)**
- Check the data for duplicates.
- Save the dataset as a csv (So that the research is replicable)
- Repeat the steps to get more data until I have about 2K for each and then combine and ger rid of duplicates and then work on the actual notebook

In [12]:
df1 = pd.read_csv('../p3_data/research.csv')

In [8]:
#Imports
import praw
import os
import pandas as pd
import requests
import time
import numpy as np


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from bs4 import BeautifulSoup

#Needed to read in the varibles that are sensitive.
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
#Initialize PRAW and Authenticate API
reddit = praw.Reddit(
    client_id = os.getenv('client_id'),
    client_secret = os.getenv('client_secret'),
    user_agent = 'text_classifier:v1.0 (by DvisionaryS)',
    username = 'DvisionaryS',
    password = os.getenv('password')
)

In [3]:
# Assign a variable to the subreddits of your choosing
subreddit_1 = 'dogs'
subreddit_2 = 'personalfinance'

In [7]:
# Function to find 1000 post for the dataset
def fetch_posts(subreddit_name, total_posts=5000, batch_size=1000):
    subreddit = reddit.subreddit(subreddit_name)  # Access the subreddit
    posts = []
    after = None  # Placeholder for pagination
    fetched_post_ids = set()  # Set to track already fetched post IDs
    loops = total_posts // batch_size  # Number of loops required, adjusted to total posts

    print(f"Fetching posts from subreddit: {subreddit_name}")

    for i in range(loops):
        if i % 10 == 0: 
            print(f"Starting loop {i + 1}/{loops}")  # Print progress every 10 loops

        try:
            # Fetch posts in batches
            batch_posts = subreddit.new(limit=batch_size)
            for post in batch_posts:
                if post is not None:
                    # Skip posts that have already been fetched
                    if post.id not in fetched_post_ids:
                        posts.append({
                            'subreddit': post.subreddit.display_name,  # Export the titles I am interested in
                            'title': post.title,
                            'selftext': post.selftext,
                            'utc': post.created_utc
                        })
                        fetched_post_ids.add(post.id)  # Mark this post as fetched

            # Simulate delay to avoid rate limits
            time.sleep(60)

        except Exception as e:
            print(f"Error occurred: {e}")
            break

    return pd.DataFrame(posts)

In [5]:
#Fetch the post from each subreddits
df_dogs = fetch_posts(subreddit_1)
df_finance = fetch_posts(subreddit_2)

Fetching posts from subreddit: dogs
Starting loop 1/5
Done fetching posts!
Number of posts received: 947
Fetching posts from subreddit: personalfinance
Starting loop 1/5
Done fetching posts!
Number of posts received: 985


In [6]:
df_dogs.head()

,subreddit,title,selftext,utc
0,dogs,Megathread: Pet Insurance,Which pet insurance is best for you? Are there...,1.732469e+09
1,dogs,Megathread: Why Does My Dog Do That?,Does your dog turn his head when he hears an o...,1.732468e+09
2,dogs,Need some advice working situation and my Catt...,\nSo Remi is almost 5 years old and a few mont...,1.732465e+09
3,dogs,"Megathread: Aging, Illness, and Euthanasia Sup...",This thread is where to get emotional support ...,1.732464e+09
4,dogs,Miss my doggo,15 years a cute ass lab and shit ton of memori...,1.732391e+09


In [9]:
df_finance.head()

,subreddit,title,selftext,utc
0,personalfinance,How do I make real money?,I was talking to a friend of mine yesterday ab...,1.732494e+09
1,personalfinance,I’m Not Enjoying Life Til I’m Out Of Debt - 24...,2 Years ago I was in a good financial situatio...,1.732494e+09
2,personalfinance,Should I pay off my mortgage?,My spouse and I (mid-30s) have a mortgage bala...,1.732493e+09
3,personalfinance,Should I do backdoor Roth?,"Hi Everyone! 33 M here, married, no kids. We a...",1.732493e+09
4,personalfinance,How do you account for gifts and one-time purc...,"When it comes to your budget, **how do you acc...",1.732493e+09


In [10]:
df = pd.read_csv('./Data/reddit_combined_df.csv')

In [12]:
df.head()

,subreddit,title,selftext,utc
0,dogs,Best pet friendly couches,I am looking to replace my couches and looking...,1.732064e+09
1,dogs,Hello beautiful dog owners!,I got offered a Pembroke puppy. She’s lower p...,1.732061e+09
2,dogs,best dog breed for a nervous cat?,hi! hope im doing this right lol i dont use re...,1.732055e+09
3,dogs,Can I train my dog to like soft toys without r...,I have the sweetest 5 y/o Newfie with quite a ...,1.732039e+09
4,dogs,Megathread: Why Does My Dog Do That?,Does your dog turn his head when he hears an o...,1.732036e+09


In [13]:
all_posts = pd.concat([df_dogs, df_finance], ignore_index=True)

In [14]:
all_posts.head()

,subreddit,title,selftext,utc
0,dogs,Megathread: Pet Insurance,Which pet insurance is best for you? Are there...,1.732469e+09
1,dogs,Megathread: Why Does My Dog Do That?,Does your dog turn his head when he hears an o...,1.732468e+09
2,dogs,Need some advice working situation and my Catt...,\nSo Remi is almost 5 years old and a few mont...,1.732465e+09
3,dogs,"Megathread: Aging, Illness, and Euthanasia Sup...",This thread is where to get emotional support ...,1.732464e+09
4,dogs,Miss my doggo,15 years a cute ass lab and shit ton of memori...,1.732391e+09


In [16]:
all_posts.shape

(1932, 4)

In [15]:
total = pd.concat([all_posts, df], ignore_index=True)

In [17]:
total.shape

(9642, 4)

In [23]:
duplicates = total[total.duplicated(subset=['utc'])]
duplicates.shape

(6819, 4)

In [24]:
df_no_duplicates = total.drop_duplicates(subset=['utc'])

In [26]:
df_no_duplicates.head()

,subreddit,title,selftext,utc
0,dogs,Megathread: Pet Insurance,Which pet insurance is best for you? Are there...,1.732469e+09
1,dogs,Megathread: Why Does My Dog Do That?,Does your dog turn his head when he hears an o...,1.732468e+09
2,dogs,Need some advice working situation and my Catt...,\nSo Remi is almost 5 years old and a few mont...,1.732465e+09
3,dogs,"Megathread: Aging, Illness, and Euthanasia Sup...",This thread is where to get emotional support ...,1.732464e+09
4,dogs,Miss my doggo,15 years a cute ass lab and shit ton of memori...,1.732391e+09


In [27]:
df1 = df_no_duplicates

In [29]:
df1.shape

(2823, 4)

In [30]:
df1.to_csv('research.csv',index=False)

In [32]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2823 entries, 0 to 8837
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   subreddit  2823 non-null   object 
 1   title      2823 non-null   object 
 2   selftext   2821 non-null   object 
 3   utc        2823 non-null   float64
dtypes: float64(1), object(3)
memory usage: 110.3+ KB


In [13]:
df = df1[['title','selftext','subreddit','utc']]

In [14]:
df.head()

,title,selftext,subreddit,utc
0,Megathread: Pet Insurance,Which pet insurance is best for you? Are there...,dogs,1.732469e+09
1,Megathread: Why Does My Dog Do That?,Does your dog turn his head when he hears an o...,dogs,1.732468e+09
2,Need some advice working situation and my Catt...,\nSo Remi is almost 5 years old and a few mont...,dogs,1.732465e+09
3,"Megathread: Aging, Illness, and Euthanasia Sup...",This thread is where to get emotional support ...,dogs,1.732464e+09
4,Miss my doggo,15 years a cute ass lab and shit ton of memori...,dogs,1.732391e+09


In [15]:
#Check for null values
df.isnull().sum()

title         0
selftext     11
subreddit     0
utc           0
dtype: int64

In [16]:
#Check for blank strings
def blanks(df):
    print((df[df.columns] == '').sum())

blanks(df)

title        0
selftext     0
subreddit    0
utc          0
dtype: int64


In [17]:
#Fill with nas so we can drop the blanks as there is only 9
df['selftext'].replace('', np.nan, inplace=True)
df.dropna(inplace=True)
blanks(df)

title        0
selftext     0
subreddit    0
utc          0
dtype: int64


/var/folders/pk/z58dkjgn50153_74d2055v0r0000gn/T/ipykernel_50645/2275500759.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['selftext'].replace('', np.nan, inplace=True)
/var/folders/pk/z58dkjgn50153_74d2055v0r0000gn/T/ipykernel_50645/2275500759.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['selftext'].replace('', np.nan, inpla

In [18]:
subreddit_counts=df['subreddit'].value_counts()

In [19]:
#Get the lowest subreddit count and downsample so they are even
min_count = min(subreddit_counts)
df_balanced = df.groupby('subreddit').apply(lambda x: x.sample(min_count, random_state=42))

/var/folders/pk/z58dkjgn50153_74d2055v0r0000gn/T/ipykernel_50645/1605757810.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df.groupby('subreddit').apply(lambda x: x.sample(min_count, random_state=42))


In [20]:
#Reset index after balancing
df_balanced = df_balanced.reset_index(drop=True)

In [21]:
#dropped the excess so that the data is even for modeling accuracy
df_balanced['subreddit'].value_counts()

subreddit
dogs               1002
personalfinance    1002
Name: count, dtype: int64

In [22]:
df_balanced.to_csv('project_cdf.csv',index=False)

In [23]:
from sklearn.utils import shuffle
df_balanced = shuffle(df_balanced)

### Map and replace personalfinance category to 0 and dogs to 1

In [24]:
df_balanced['subreddit'] = df_balanced['subreddit'].map(dict(dogs=1, personalfinance=0))

In [25]:
df_balanced['subreddit'].value_counts()

subreddit
0    1002
1    1002
Name: count, dtype: int64

In [26]:
df_balanced.head()

,title,selftext,subreddit,utc
1908,"Living overseas, investing in the US...worried...","Hi everyone, I'm living overseas at the moment...",0,1.731653e+09
831,"[Weekly Vent]September 30-04, 2024",Is someone not picking up poop in front of you...,1,1.727698e+09
278,Choosing My First Dog: Shih Tzu vs. Bichon Fri...,"Hello, r/dogs community! I’m on the verge of b...",1,1.731888e+09
1843,Early withdrawal after termination with outsta...,I’m going to go over what I think I know. I’m ...,0,1.732383e+09
584,Megathread: Why Does My Dog Do That?,Does your dog turn his head when he hears an o...,1,1.727539e+09


In [27]:
df_balanced.to_csv('final_df.csv',index=False)

## Data Collection up to the final csv

In [29]:
#View the data
#Saved this data as a csv with my first pulls
sub_1.head()

,subreddit,title,selftext,utc
0,dogs,Best pet friendly couches,I am looking to replace my couches and looking...,1.732064e+09
1,dogs,Hello beautiful dog owners!,I got offered a Pembroke puppy. She’s lower p...,1.732061e+09
2,dogs,best dog breed for a nervous cat?,hi! hope im doing this right lol i dont use re...,1.732055e+09
3,dogs,Can I train my dog to like soft toys without r...,I have the sweetest 5 y/o Newfie with quite a ...,1.732039e+09
4,dogs,Megathread: Why Does My Dog Do That?,Does your dog turn his head when he hears an o...,1.732036e+09


In [30]:
#View the amount of data
sub_1.shape

(949, 4)

In [34]:
#Check for duplicates
sub_1['utc'].value_counts()

utc
1.732064e+09    1
1.728277e+09    1
1.728311e+09    1
1.728302e+09    1
1.728299e+09    1
               ..
1.730150e+09    1
1.730149e+09    1
1.730144e+09    1
1.730142e+09    1
1.726435e+09    1
Name: count, Length: 949, dtype: int64

In [35]:
#View the data
sub_2.head()

,subreddit,title,selftext,utc
0,personalfinance,Credit card 0% purchase / balance transfer,As title suggests I have received this card wi...,1.732110e+09
1,personalfinance,ATM didn’t give me my cash,so I recently used an out-of-network ATM at a ...,1.732110e+09
2,personalfinance,Teacher retirement: Is this enough?,I am in my 21st year as a teacher and gross 95...,1.732108e+09
3,personalfinance,Should I get a new credit card?,I'm moving to New York City in a couple weeks....,1.732108e+09
4,personalfinance,More salary or more retirement,Hypothetical \n\nMaxing out retirement so this...,1.732107e+09


In [36]:
#View the amount of data
sub_2.shape

(982, 4)

In [37]:
#Check for duplicates
sub_2['utc'].value_counts()

utc
1.731691e+09    2
1.731947e+09    2
1.732110e+09    1
1.731762e+09    1
1.731771e+09    1
               ..
1.731956e+09    1
1.731956e+09    1
1.731956e+09    1
1.731956e+09    1
1.731603e+09    1
Name: count, Length: 980, dtype: int64

In [40]:
#Duplicates have been Identifed so drop them.
#Add to notes why we dropped the dupplicates. [After cleaning remove this]
sub_2 = sub_2.drop_duplicates(subset=['utc'])

In [41]:
sub_2['utc'].value_counts()

utc
1.732110e+09    1
1.731772e+09    1
1.731771e+09    1
1.731769e+09    1
1.731769e+09    1
               ..
1.731956e+09    1
1.731956e+09    1
1.731956e+09    1
1.731956e+09    1
1.731603e+09    1
Name: count, Length: 980, dtype: int64

In [42]:
#Save the dataset
sub_1.to_csv('data/subreddit_1_dogposts.csv', index=False)
sub_2.to_csv('data/subreddit_2_financeposts.csv', index=False)

#### Combine all of the data we pulled
- Read in the data
- Assign the variable to the data
- Combine the data
- Save the combine dataset as 1

In [19]:
df1 = pd.read_csv('./Data/subreddit_1_dogposts.csv')

df_1 = pd.read_csv('./Data/subreddit_1_dogposts1.csv')

df__1 = pd.read_csv ('./Data/subreddit_1_dogposts2.csv')

df1_ = pd.read_csv ('./Data/subreddit_1_dogposts3.csv')

df2 = pd.read_csv('./Data/subreddit_2_financeposts.csv')

df_2 = pd.read_csv('./Data/subreddit_2_financeposts1.csv')

df__2 = pd.read_csv('./Data/subreddit_2_financeposts2.csv')

df2_ = pd.read_csv('./Data/subreddit_2_financeposts3.csv')

In [20]:
reddit_df = pd.concat([df1, df_1, df__1, df1_, df2, df_2, df__2, df2_], axis=0, ignore_index=True)

In [31]:
reddit_df.to_csv('./Data/reddit_combined_df.csv', index=False)

In [22]:
reddit_df.head()

,subreddit,title,selftext,utc
0,dogs,Best pet friendly couches,I am looking to replace my couches and looking...,1.732064e+09
1,dogs,Hello beautiful dog owners!,I got offered a Pembroke puppy. She’s lower p...,1.732061e+09
2,dogs,best dog breed for a nervous cat?,hi! hope im doing this right lol i dont use re...,1.732055e+09
3,dogs,Can I train my dog to like soft toys without r...,I have the sweetest 5 y/o Newfie with quite a ...,1.732039e+09
4,dogs,Megathread: Why Does My Dog Do That?,Does your dog turn his head when he hears an o...,1.732036e+09


In [23]:
reddit_df['subreddit'] == 'personalfinance'

0       False
1       False
2       False
3       False
4       False
        ...  
7705     True
7706     True
7707     True
7708     True
7709     True
Name: subreddit, Length: 7710, dtype: bool

In [25]:
reddit_df['utc'].value_counts()

utc
1.732064e+09    4
1.727032e+09    4
1.726954e+09    4
1.726966e+09    4
1.726970e+09    4
               ..
1.731676e+09    1
1.731676e+09    1
1.731675e+09    1
1.731674e+09    1
1.732223e+09    1
Name: count, Length: 2642, dtype: int64

In [28]:
red_df = reddit_df.drop_duplicates(subset=['utc'])

In [29]:
red_df['utc'].value_counts()

utc
1.732064e+09    1
1.731699e+09    1
1.731696e+09    1
1.731696e+09    1
1.731696e+09    1
               ..
1.726706e+09    1
1.726701e+09    1
1.726696e+09    1
1.726690e+09    1
1.732223e+09    1
Name: count, Length: 2642, dtype: int64

In [30]:
red_df.to_csv('data/final_reddit_data.csv', index=False)

In [22]:
sub_2 = sub_2.drop_duplicates(subset=['utc'])

In [24]:
sub_2['utc'].value_counts()

utc
1.732200e+09    1
1.731878e+09    1
1.731878e+09    1
1.731876e+09    1
1.731875e+09    1
               ..
1.732028e+09    1
1.732035e+09    1
1.732035e+09    1
1.732034e+09    1
1.731684e+09    1
Name: count, Length: 979, dtype: int64

In [25]:
sub_1.to_csv('data/subreddit_1_dogposts1.csv', index=False)
sub_2.to_csv('data/subreddit_2_financeposts1.csv', index=False)

In [14]:
sub_1.shape

(949, 4)

In [17]:
sub_1['utc'].value_counts()

utc
1.732161e+09    1
1.728321e+09    1
1.728350e+09    1
1.728346e+09    1
1.728346e+09    1
               ..
1.730261e+09    1
1.730262e+09    1
1.730250e+09    1
1.730243e+09    1
1.726465e+09    1
Name: count, Length: 949, dtype: int64

In [18]:
sub_2.head()

,subreddit,title,selftext,utc
0,personalfinance,Roth IRA contributions to 403(b),My organizations open enrollment period just o...,1.732200e+09
1,personalfinance,Locating IRA or 401k of Estate,My wife has become the executor of a deceased ...,1.732199e+09
2,personalfinance,Should I put my savings in an IRA?,Hi! I am new to personal finance and am trying...,1.732199e+09
3,personalfinance,Inherited mutual funds. Thinking of selling an...,Hi all - \nHopefully an appropriate question. ...,1.732199e+09
4,personalfinance,Double-checking my understanding of retirement...,I have a 401k through work with a 4% employer ...,1.732199e+09


In [19]:
sub_2.shape

(981, 4)

In [21]:
sub_2['utc'].value_counts()

utc
1.731947e+09    2
1.731691e+09    2
1.731862e+09    1
1.731868e+09    1
1.731878e+09    1
               ..
1.732028e+09    1
1.732035e+09    1
1.732035e+09    1
1.732034e+09    1
1.731684e+09    1
Name: count, Length: 979, dtype: int64

In [11]:
sub_2['utc'].value_counts()

utc
1.731947e+09    2
1.732221e+09    2
1.732011e+09    1
1.732025e+09    1
1.732024e+09    1
               ..
1.732154e+09    1
1.732153e+09    1
1.732153e+09    1
1.732152e+09    1
1.731783e+09    1
Name: count, Length: 979, dtype: int64

In [12]:
sub_2 = sub_2.drop_duplicates(subset=['utc'])

In [13]:
sub_2['utc'].value_counts()

utc
1.732305e+09    1
1.732026e+09    1
1.732025e+09    1
1.732024e+09    1
1.732023e+09    1
               ..
1.732156e+09    1
1.732154e+09    1
1.732153e+09    1
1.732153e+09    1
1.731783e+09    1
Name: count, Length: 979, dtype: int64

In [14]:
sub_1.to_csv('data/subreddit_1_dogposts2.csv', index=False)
sub_2.to_csv('data/subreddit_2_financeposts2.csv', index=False)

In [12]:
#Second fetch to combine with 1st taking out the ones that were repeats
sub_1.head()

,subreddit,title,selftext,utc
0,dogs,[breed] recommendation for a lifelong cat lady,I have been a cat lady my whole life. When I m...,1.732161e+09
1,dogs,Cheap stocking stuffers?,"Hi, so for Christmas I'm wanting to get my dog...",1.732200e+09
2,dogs,What dog should I have?,"Hi! I’m considering getting a dog, but I need ...",1.732194e+09
3,dogs,Can I get a dog?,"1.Will this be your first dog? If not, what ex...",1.732190e+09
4,dogs,18 month old puppy barks at window for attenti...,He won't stop barking at the window at bed tim...,1.732184e+09


In [6]:
#Third fetch to combine with 1st taking out the ones that were repeats
sub_1.head()

,subreddit,title,selftext,utc
0,dogs,Breed Suggestions,\*\*Introduction\*\* \n \n1)Will this be you...,1.732304e+09
1,dogs,Best donut dog bed from Amazon?,Is there a donut dog bed from Amazon worth get...,1.732297e+09
2,dogs,What dog would be good for an active lifestyle...,\*\*Introduction\*\*\n\n**1) Will this be your...,1.732299e+09
3,dogs,Looking for new and creative enrichment ideas,I am constantly trying to find new enrichment ...,1.732298e+09
4,dogs,How to stop my dog from biting my cat?,My dog has a thing for biting at my cat consta...,1.732296e+09


In [7]:
sub_1.shape

(948, 4)

In [8]:
sub_1['utc'].value_counts()

utc
1.732304e+09    1
1.728390e+09    1
1.728418e+09    1
1.728416e+09    1
1.728416e+09    1
               ..
1.730315e+09    1
1.730309e+09    1
1.730304e+09    1
1.730301e+09    1
1.726501e+09    1
Name: count, Length: 948, dtype: int64

In [9]:
sub_2.head()

,subreddit,title,selftext,utc
0,personalfinance,Need easy access to a personal loan to pay off...,I want to get access to a personal loan to pay...,1.732305e+09
1,personalfinance,Can you remove advisor on a 529 plan?,I have a 529 plan for my child. It was set up ...,1.732304e+09
2,personalfinance,Long-term inflation versus mortgage interest c...,Is there a handy calculator out there that hel...,1.732304e+09
3,personalfinance,Pay for car cash or let it ride in the market?,Looking for advice since I've asked around and...,1.732304e+09
4,personalfinance,Scammed by a locksmith?,I accidentally lost my key walking my dog and ...,1.732304e+09


In [10]:
sub_2.shape

(981, 4)

In [10]:
sub_2.shape

(979, 4)

In [11]:
sub_2['utc'].value_counts()

utc
1.732221e+09    2
1.732090e+09    1
1.732105e+09    1
1.732104e+09    1
1.732101e+09    1
               ..
1.732237e+09    1
1.732236e+09    1
1.732236e+09    1
1.732235e+09    1
1.731950e+09    1
Name: count, Length: 978, dtype: int64

In [12]:
sub_2 = sub_2.drop_duplicates(subset=['utc'])

In [13]:
sub_2['utc'].value_counts()

utc
1.732405e+09    1
1.732105e+09    1
1.732105e+09    1
1.732104e+09    1
1.732101e+09    1
               ..
1.732237e+09    1
1.732236e+09    1
1.732236e+09    1
1.732235e+09    1
1.731950e+09    1
Name: count, Length: 978, dtype: int64

In [14]:
sub_1.to_csv('data/subreddit_1_dogposts3.csv', index=False)
sub_2.to_csv('data/subreddit_2_financeposts3.csv', index=False)

In [7]:
sub_1.shape

(948, 4)

In [8]:
sub_1['utc'].value_counts()

utc
1.732391e+09    1
1.728407e+09    1
1.728480e+09    1
1.728476e+09    1
1.728470e+09    1
               ..
1.730309e+09    1
1.730301e+09    1
1.730295e+09    1
1.730315e+09    1
1.726512e+09    1
Name: count, Length: 948, dtype: int64

In [9]:
sub_2.head()

,subreddit,title,selftext,utc
0,personalfinance,How do you cope with losing money,"Long story short, I gambled and lost 90% of my...",1.732405e+09
1,personalfinance,Are Black Friday Deals Worth the Wait?,"Hi everyone, \n\nI am a college student who is...",1.732404e+09
2,personalfinance,HSA Contribution limits 2 income household,"So, up until this year my family (spouse + 2 k...",1.732404e+09
3,personalfinance,Paying lump sum payment on auto loan to reduce...,Outline of situation\n\n- Purchased new vehicl...,1.732404e+09
4,personalfinance,How to Handle Large Daycare Assistance Gift,My wife and I are expecting our first child in...,1.732404e+09


In [6]:
# Forth fetch to combine with 1st taking out the ones that were repeats
sub_1.head()

,subreddit,title,selftext,utc
0,dogs,Miss my doggo,15 years a cute ass lab and shit ton of memori...,1.732391e+09
1,dogs,Which breed do you think is most ball-obsessed?,Just for fun! Been wondering this lately 🤪\n\n...,1.732389e+09
2,dogs,Hard licky mat,Does anyone know anywhere that does a lucky Ma...,1.732382e+09
3,dogs,Megathread: Why Does My Dog Do That?,Does your dog turn his head when he hears an o...,1.732381e+09
4,dogs,"Megathread: Aging, Illness, and Euthanasia Sup...",This thread is where to get emotional support ...,1.732378e+09
